### TP 3-4 Julie Chapdelaine et Cleo Daguin
Le but de ce TP est de se familiariser avec les réseaux neuronaux réccurents et de travailler sur de la classification de textes.

#### **I) Reccurent neural network : IMBD sentiment classification**

Nous allons d'abord commencer par étudier le réseau LSTM appliqué à un commentaire de film pour qu'il indique si celui ci est positif ou négatif.
Source du code : https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.datasets import imdb

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Using TensorFlow backend.


2.2.0-rc2


Initialisation des constantes du réseau avec le nombre de mots différents pris en compte dans le réseau, la taille des entrées et le nombre d'éléments pris en compte à chaque itération du model.

In [0]:
## Nombre de mots pris en compte
max_features = 20000
## Taille des entrées
maxlen = 80
## Nombre d'éléments pour chaque epoch du model
batch_size = 32

**Téléchargement des données et séparation** entre les données d'entrainement (train) et de test (test) et entre les entrées (x) et les sorties(y).
Les données téléchargées sont sous la forme d'un tableau d'indices des mots du commentaire et d'un chiffre binaire qui indique si le commentaire est positif ou négatif. Les mots sont enregistrés dans un dictionnaire et les indices correspondent au classement des mots en fonction de leur fréquence : le mot avec l'indice 1 est le mot le plus courant. 

In [0]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


Utilisation de pad_sequence pour que toutes les entrées aient la même taille maxlen définie plus haut soit en troncant si la taille de l'entrée est plus grande soit en rajoutant des valeurs neutres.

In [0]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


**Création du model.**

*   Utilisation de **Embedding** pour faire du plongement lexical soit ne plus définir chaque mot par un nombre (car cela ne permet pas de faire des recoupements entre mots différents mais proches) mais par un vecteur en fonction de son contexte. Ainsi on peut trouver des similitudes entre certains mots. 
    *   Le premier argument de la fonction indique qu'on ne va prendre en compte qu'un certain nombre de mots qui vont etre les plus courants dans le dictionnaire (ici les 2000 premiers) et remplacer les autres par une valeur neutre (ici 0). Ainsi, il y a moins de mots à traiter donc un traitement plus rapide et moins le mot est courant moins il sera présent et donc moins il sera utile pour savoir si le commentaire est négatif ou positif. 
    *   Le deuxième paramètre indique la taille du vecteur qui va représenter le mot.
*   Utilisation de **LSTM** car il y a beaucoup d'informations différentes passées en entrée et peu sont utiles. LSTM va permettre de faire le tri entre les informations et ne garder que les utiles alors qu'un réseau neuronal plus classique va utiliser toutes les informations et va etre surchargé. Le premier argument est la taille et dépend directement de l'argument fourni dans l'embedding. Le dropout va mettre certains poids du réseau à 0 et ainsi éviter de coller trop à l'exemple (overfitting).
*   Utilisation de **sigmoid** car il n'y a que 2 sorties, 0 et 1 pour un commentaire négatif ou positif.
*   Ce qui change le plus ici par rapport au model du TP précédent sur la même base de données est qu'ici on prend en compte l'ordre dans lequel les mots sont placés les uns par rapport aux autres ce qui va donner un résultat plus sûr mais aussi plus de calculs.

In [0]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Build model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


**Compilation du model**
*    Utilisation de **binary_crossentropy** car utilisation de sigmoid plus haut.

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**Application du réseau** sur les données de training


In [0]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...



Train on 25000 samples, validate on 25000 samples
Epoch 1/3





25000/25000 [==============================] - 161s 6ms/step - loss: 0.4546 - acc: 0.7859 - val_loss: 0.3779 - val_acc: 0.8356
Epoch 2/3
25000/25000 [==============================] - 160s 6ms/step - loss: 0.3036 - acc: 0.8776 - val_loss: 0.3869 - val_acc: 0.8336
Epoch 3/3
25000/25000 [==============================] - 22s 877us/step
Test score: 0.41466219365119933
Test accuracy: 0.83404


Training loss is smaller than test loss. It means that our network is likely to overfit. To prevent it, we will increase the dropout.

**Modification des paramètres**

*   Augmentation du dropout. Ainsi, certains poids seront mis à 0 et cela réduira le risque de trop coller aux données d'entrainement et de faire un overfitting




In [0]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.8, recurrent_dropout=0.6))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 136s 5ms/step - loss: 0.5748 - accuracy: 0.6956 - val_loss: 0.4258 - val_accuracy: 0.8152
Epoch 2/3
25000/25000 [==============================] - 135s 5ms/step - loss: 0.4444 - accuracy: 0.7997 - val_loss: 0.3938 - val_accuracy: 0.8270
Epoch 3/3
25000/25000 [==============================] - 22s 890us/step
Test score: 0.39081519746780397
Test accuracy: 0.8334800004959106


On peut voir que la val_loss diminue à chaque epoch. L'accuracy n'est qu'a 83 % mais elle augment bien à chaque epoch et pour avoir une meilleure accuracy, il suffit juste de faire plus d'epochs, ce que nous n'avons pas fait car c'est un processus très long.

#### **II) Text classification: the Ohsumed dataset**
Nous allons suivre les mêmes étapes pour entrainer un réseau de neurones réccurents qui détermine parmi 23 catégories le sujet d'un article médical.

Initialisation des constantes

In [0]:
## Nombre de mots pris en compte
max_features = 20000
## Taille des entrées
maxlen = 80
## Nombre d'éléments pour chaque epoch du model
batch_size = 32

Récupération et traitement des données.

In [0]:
def get_info(path: str):
    data = list(os.walk(path))[1:]
    files = []
    for d in data:
        folder_name = d[0]
        for file in d[2]:
            files.append((folder_name.split('/')[-1], os.path.join(folder_name, file)))

    d = defaultdict(int)
    texts = defaultdict(list)
    for (cate, file) in files:
        with open(file, 'r') as outfile:
            text = outfile.read()
            texts[cate].append(text)
            words = text_to_word_sequence(text)
            for word in words:
                d[word] += 1
    words = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return (texts, words)

def load_data(texts, words_index):
    tokenizer = RegexpTokenizer(r'\w+')
    index_word_start = 20
    index_word_end = 10000
    x = []
    y = []
    step = 0

    for categorie, articles in texts.items():
        for article in articles:
            article_indexed = []
            tokens = tokenizer.tokenize(article)
            for token in tokens:
                index_token = 0
                try:
                    index_token = 1 + words_index[index_word_start:index_word_end].index(token)
                except:
                    pass
                article_indexed.append(index_token)
            x.append(article_indexed)
            y.append(list(categories.keys()).index(categorie))
    
    return (np.array(x), np.array(y))

In [0]:
texts_train, words_train = get_info('./ohsumed-first-20000-docs/training')
texts_test, words_test = get_info('./ohsumed-first-20000-docs/test')

words_index = []
for word, count in words_train:
    words_index.append(word)

(x_train, y_train) = load_data(texts_train, words_index)
(x_test, y_test) = load_data(texts_test, words_index)

Utilisation de pad sequence

In [0]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

##### Création du model
La majeur différence ici est du au fait qu'ilne s'agisse plus d'une sortie binaire mais de multiples sorties binaires. Le nombre de sorties qui doit etre égale au nombre de catégories. On a également ajouter une couche de neurones pour un apprentissage plus fin, ce qui est logique avec une augmentation des sorties. On a aussi rajouter un dropout pour éviter l'overfitting.

In [0]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(23, activation='sofmax'))

Compilation et entrainement

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)